# EDS-TEM quantification of core shell nanoparticles

This notebook is based on the [EDS-TEM analysis demo by Pierre Burdet](https://github.com/hyperspy/hyperspy-demos/blob/main/electron_microscopy/EDS/TEM_EDS_nanoparticles.ipynb). It shows how to perform basic EDS analysis with HyperSpy.

## Data and sample description

The sample and the data used in this tutorial are described in 
D. Roussow, et al., Nano Letters, In Press (2015) (see the [full article](https://www.repository.cam.ac.uk/bitstream/handle/1810/248102/Roussouw%20et%20al%202015%20Nano%20Letters.pdf?sequence=1)).

FePt@Fe$_3$O$_4$ core-shell nanoparticles are investigated with an EDS/TEM experiment (FEI Osiris TEM, 4 EDS detectors). The composition of the core can be measured with ICA (see figure 1c). To prove the accuracy of the results, measurements on bare FePt bimetallic nanoparticles from a synthesis prior to the shell addition step are used.

<img src="images/core_shell.png" style="height:350px;">
Figure 1: (a) A spectrum image obtained from a cluster of core-shell nanoparticles. (b) The nanoparticles are comprised of a bi-metallic Pt/Fe core surrounded by an iron oxide shell on a carbon support. (c) ICA decomposes the mixed EDX signals into components representing the core (IC#0), shell (IC#1) and support (IC#2).

To download the data required for this tutorial execute the cell below:

In [ ]:
from urllib.request import urlretrieve, urlopen
from zipfile import ZipFile
files = urlretrieve("https://www.dropbox.com/s/ecdlgwxjq04m5mx/HyperSpy_demos_EDS_TEM_files.zip?raw=1", "./HyperSpy_demos_EDX_TEM_files.zip")
with ZipFile("HyperSpy_demos_EDX_TEM_files.zip") as z:
    z.extractall()

## Loading and viewing data

Import HyperSpy

*Remember, if at any point you do not understand how a function operates, its help file can be loaded by typing the name of the command followed by a '?' into a cell and then running that cell.*

In [ ]:
%matplotlib qt
import hyperspy.api as hs

Let's load the spectrum image (SI) of the core-shell particles:

In [ ]:
cs = hs.load("core_shell.hdf5")

Check the metadata has imported correctly. In particular whether the list of elements you wish to analyse is correct.

In [ ]:
cs.metadata

Plot the core-shell data to inspect the signal level:

In [ ]:
cs.plot()

Plotting the integrated counts for the whole spectrum image is a good way to check what elements exist in the sample. Adding 'True' to the function also labels any elements from the metadata onto the spectrum.

In [ ]:
cs.sum().plot(xray_lines=True)

## Basic EDS analysis

### Extracting count maps of elements

If they're not already added it is important to make sure all the elements you want to extract the intensities for are in the metadata of the sample.

In [ ]:
cs.set_elements(['Fe','Pt'])
cs.set_lines(['Fe_Ka', 'Pt_La'])



Extracting lines can be done without any background or integration window parameters. However if none are specified the default integration window is 1 FWHM and no background subtraction is carried out.

Line_width is the distance from the x-ray line (in FWHM) the the background window is taken [left, right] allowing different distances for the two directions.
An asymmetric value is used here because otherwise the Pt background windows overlap with the Cu K$_β$ line from the sample grid.

In [ ]:
bw = cs.estimate_background_windows(line_width=[5.0, 2.0])
iw =  cs.estimate_integration_windows(windows_width=3)

It is important to plot the windows to check that they are selecting the data correctly otherwise errors, particularly in background subtraction arise.

The integration windows are represented by dashed lines and background windows by solid lines. The estimated background is the plotted by the close to horizontal black lines.

In [ ]:
cs.sum().plot(True, background_windows=bw, integration_windows=iw)

*Try running the previous two cells of code  above with line_width=[3.0,3.0] and see how this results in an erroneous, background subtraction by plotting the background lines. (You might need to zoom in to see it)*

How accurate background subtraction will be on a pixel-by-pixel basis can be see with this plot. 

The x and y sliders select a pixel in the particle images we plotted earlier. 

You should be able to find some examples (e.g. the Fe K$_α$ line at X=39, Y=44) of where the background subtraction still fails due to a poor signal-to-noise ratio in the data.



In [ ]:
cs.plot(True, background_windows=bw, navigator='slider')

Another way to adjust the location of the background windows is by changing specific numbers in the background window array individually.

Running the 'bw' command will output the array, which contains keV coordinates corresponding to the position of the background windows. Each row corresponds to a different element in the list given in the metadata. Remember arrays in Python start at (0,0).

These two commands therefore alter the position of the start and end points of the left-hand background window for Pt.

In [ ]:
bw[1, 0] = 8.44
bw[1, 1] = 8.65
bw

The background substraction method is not reliable when the number of counts is very low. Therefore, when possible and necessary, it is good to rebin the data. This can be easily done with the rebin.

The following commands perform rebinning on both the core-shell ('cs') data and the core-only ('c') data. We define using the 'scale' parameter that we want 2x binning in X, 2x binning in Y, and 1x binning in Z (our counts).

*Note, as we are re-defining 'cs' or 'c', this overwrites our previously-imported data. This means running this command multiple times will re-bin the data multiple times. If you accidentally run this command too many times, simply re-import the data by running the 'hs.load' commands at the top of this workbook'.*

In [ ]:
cs = cs.rebin(scale=(2,2,1))

Finally, once the background subtraction windows have been selected to be in careful positions it is possible to extract the intensities. 

Note that exactly the same windows have been used for analysis of both the 'core' and 'core-shell' data sets. This is critical here as we are comparing the two datasets.

In [ ]:
cs_intensities = cs.get_lines_intensity(background_windows=bw, integration_windows=iw)

Each 'get_lines_intensity' command will create a list of images, again in the same order of the list of elements in the list of metadata. If the element is not in the metadata its intensity map will not be extracted.

We can then run 'cs_intensities' to confirm the that we have extracted intensity maps for all our elements of interest.

In [ ]:
cs_intensities

In [ ]:
# Plotting one particular image (in this case, the first, Fe_Ka map) can be done with:
cs_intensities[0].plot()

All the intensity maps can be plotted using:

In [ ]:
hs.plot.plot_images(
    cs_intensities,
    axes_decor=None,
    scalebar="all",
)

Plotting and extracting intensity for both data sets can be condensed into one line.

We can change HyperSpy's default color map for this session as follows:

In [ ]:
hs.preferences.Plot.cmap_navigator = "magma"
hs.preferences.Plot.cmap_signal = "magma"

In [ ]:
cs_intensities[0].plot()

### Quantification

Hyperspy is able to carry out EDX quantification using k-factors 'CL', zeta-factors 'zeta', or cross_sections 'cross_sections'. 

All these methods are applied in the same way using the combination of the stack of intensities and and original data.

For 'zeta' or 'cross_section' quantification both a 'live_time' and a 'beam_current' should be in the metadata.

To set them:

In [ ]:
cs.set_microscope_parameters(
    live_time = 6.15, # in seconds
    beam_current = 0.5, # in nA
    beam_energy=200, # in keV
)

We also need the k-factors for iron and platinum. We can obtain experimentally from standards. In this case we take them from the Brucker Esprit software:

In [ ]:
factors = [1.450226, 5.75602]

In [ ]:
quant = cs.quantification(
    cs_intensities,
    method='CL',
    factors=factors,
)

The `quantification` method returns a list of images with the atomic percent of each element: 

In [ ]:
quant

(When quantifying using the 'zeta' and 'cross_section' methods, the method outpus more signals. See the [EDS quantification](http://hyperspy.org/hyperspy-doc/current/user_guide/eds.html#eds-quantification) section of the documentation for more details.)

In [ ]:
hs.plot.plot_images(quant)

Obviously it does not make sense to calculate the atomic percent of iron and platinum where there are no particles, what explains the "noisy" pixels in the figures above.

To fix this, we can create a rough mask by thresholding the iron intensity map.

In [ ]:
cs_intensities[0].get_histogram(20).plot()

The low counts peak corresponds to the places where there is not particles. A value around 15 should produce a reasonable mask for our purposes:

In [ ]:
mask = cs_intensities[0] > 15
mask.plot()

Let's fix the quantification results and plot them:

In [ ]:
quant = [_ * mask for _ in quant]
hs.plot.plot_images(quant)

The masked elemental maps reveal more clearly the core-shell nature of the particles

## Composition of the particle's core

The analysis above reveals that most of the particles consist of an iron shell on a platinum rich core. However, the basic analysis above cannot determine the composition of the core of the particles. In this subsection we will sovle this problem by two methods. The first one is experimental, and requires performing the same basic analysis on the same particles without the core. The second one consists on separating the signal of the core from the mixture using blind source separation methods (BSS). 

### EDS analysis of the bare cores

Let's load the "bare core" dataset and perform basic EDS analysis to compute the lines intensities as in the previous section:

In [ ]:
c = hs.load("bare_core.hdf5")
c.set_elements(['Fe','Pt'])
c.set_lines(['Fe_Ka', 'Pt_La'])
c = c.rebin(scale=(2,2,1))
c_intensities = c.get_lines_intensity(background_windows=bw, integration_windows=iw)

To obtain an integrated representative spectrum of the bare nanoparticles, we use thresholding on the Pt L$_{\alpha}$ intensity map to select only the regions where there are particles

In [ ]:
pt_la = c.get_lines_intensity(['Pt_La'])[0]
mask_bare = pt_la > 12

In [ ]:
axes = hs.plot.plot_images(
    (
        # mask_bare, # Commented out because it doesn't work with HyperSpy v1.7.4. See https://github.com/hyperspy/hyperspy/pull/3118
        pt_la,
        pt_la * mask_bare,
    ),
    axes_decor=None,
    colorbar=None,
    label=[
        # "mask", # Commented out because it doesn't work with HyperSpy v1.7.4. See https://github.com/hyperspy/hyperspy/pull/3118
        "Pt Lα intensity",
        "Pt Lα intensity bg masked"],
)

To apply the mask, we simply multiply it with the SI:

In [ ]:
c_masked = c * mask_bare

In [ ]:
c_masked.plot()

The sum over the particles is used as a bare core spectrum.

In [ ]:
s_bare = c_masked.sum()

In [ ]:
s_bare.plot()

In [ ]:
s_bare_intensity = s_bare.get_lines_intensity(xray_lines=("Fe_Ka", "Pt_La"))
s_bare_composition = s_bare.quantification(s_bare_intensity, method="CL", factors=factors)

In [ ]:
print(f'Bare core composition from bare cores: {s_bare_composition[0].data[0]:.2f}% Fe {s_bare_composition[1].data[0]:.2f}% Pt')

### Blind source separation

We start by performing SVD decomposition. For this we need to change the data type from integer to float:

In [ ]:
cs.change_dtype('float')
cs.decomposition()

The scree plot helps determining the number of components to keep for BSS:

In [ ]:
ax = cs.plot_explained_variance_ratio()

Let's perform ICA on the first three components:

In [ ]:
cs.blind_source_separation(3)

In [ ]:
cs.plot_bss_results()

The first component corresponds to the core, the second to the shell and the third to the carbon substrate.

Let's extract the first BSS and verify if its composition is consistent with the composition of the bare particles.

In [ ]:
s_bss = cs.get_bss_factors().inav[0]

In [ ]:
s_bss_intensity = s_bss.get_lines_intensity(xray_lines=("Fe_Ka", "Pt_La"))
s_bss_composition = s_bss.quantification(s_bss_intensity, method="CL", factors=factors)

In [ ]:
s_bss_composition[0].data

In [ ]:
print(f'Bare core composition from BSS {s_bss_composition[0].data[0]:.2f}% Fe {s_bss_composition[1].data[0]:.2f}% Pt')

This is very closed to the composition of the bare cores obtained above:

In [ ]:
print(f'Bare core composition from bare cores: {s_bare_composition[0].data[0]:.2f}% Fe {s_bare_composition[1].data[0]:.2f}% Pt')